In [1]:
import pandas as pd
import numpy as np
import os, glob, sys
import matplotlib.pyplot as plt


import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

from tqdm import tqdm

device = torch.device('cuda:{}'.format(0) if torch.cuda.is_available() else 'cpu')
torch.cuda.set_device(device) # change allocation of current GPU 

import warnings
warnings.filterwarnings(action='ignore') 

/home/user/anaconda3/envs/leakage/lib/python3.7/site-packages/torch/cuda/__init__.py:104: UserWarning: 
NVIDIA GeForce RTX 3090 with CUDA capability sm_86 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_37 sm_50 sm_60 sm_70.
If you want to use the NVIDIA GeForce RTX 3090 GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(incompatible_device_warn.format(device_name, capability, " ".join(arch_list), device_name))


In [2]:
df = pd.read_csv('/home/data/psk/leakage/ID_13.csv')
df = df[df.cycle.apply(lambda x: x in [1,3,4,5])]

col_list = ['TimeStamp', 'PM1.Gas2_Monitor', 'PM1.APC_Position',
    'PM1.SourcePwr1_Read', 'PM1.SourcePwr2_Read', 'PM1.Temp1', 'PM1.Temp2', 'WallTemp',
    'Port_Num', 'Process_Num', 'Wafer_Status', 'Folder_Name', 'File_Name', 'Torr','Leak', 'cycle']
df = df.loc[:, col_list]

wafer_unit = list(df.groupby(['Port_Num', 'Process_Num', 'Wafer_Status',
                                'Folder_Name', 'File_Name', 'cycle'])) # KEY 값
wafer_list = list(filter(lambda x: 102 > len(x[1]) > 90, wafer_unit))

In [3]:
cycle_num = df.cycle.unique()
cycle_dict = {}

cycle_mean = {}
cycle_std = {}

c_m_list = []
c_s_list = []

for cycle in tqdm(cycle_num):
    df_ = df[df.cycle == cycle]
    wafer_unit_ = list(df_.groupby(['Port_Num', 'Process_Num', 'Wafer_Status',
                                    'Folder_Name', 'File_Name', 'cycle'])) # KEY 값
    wafer_list_ = list(filter(lambda x: 102 > len(x[1]) > 90, wafer_unit_))

    w_mean = np.array([df.iloc[:, :-8].mean().values for info, df in wafer_list_])
    w_std = np.array([df.iloc[:, :-8].std().values for info, df in wafer_list_])

    c_mean = np.mean(w_mean, axis = 0)
    cycle_mean[cycle] = c_mean
    
    c_std = np.mean(w_std, axis = 0)
    cycle_std[cycle] = c_std
    
cycle_dict['mean'] = cycle_mean
cycle_dict['std'] = cycle_std

100%|██████████| 4/4 [00:26<00:00,  6.52s/it]


In [4]:
from sklearn.model_selection import train_test_split

X_train_, X_test, y_train, y_test = train_test_split(wafer_list, range(len(wafer_list)), test_size = 0.1, random_state=1234)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_, range(len(X_train_)), test_size = 0.1, random_state=1234)


In [5]:
class datasetAE(Dataset):
    def __init__(self, wafer_unit, window_size, max_len, cycle_dict):
        
        super(datasetAE, self).__init__()
        self.wafer_unit = wafer_unit
        self.window_size = window_size
        self.max_len = max_len
        self.cycle_dict = cycle_dict
    
    def __getitem__(self, idx):
        
        info = self.wafer_unit[idx][0]
        df = self.wafer_unit[idx][1]
        cycle = df.cycle.unique()[0]
    
        c_mean = self.cycle_dict['mean'][cycle]
        c_std = self.cycle_dict['std'][cycle]
        
        torr = df.Torr.unique()[0]
        y = df.Leak.unique()[0]
        
        df_ = df.iloc[:, 1:-8].reset_index(drop=True)
                
        # padding length
        if len(df_) < self.max_len:
            
            add_num = abs(len(df_) - self.max_len)
            for num in range(add_num):
                df_.loc[len(df_)] = 0
                
        array_df = np.array(df_)
        df__ = (array_df - c_mean) / (c_std)
        
        x = np.array(df__).T
        
        return x, np.array(y), torr
    
    def __len__(self):
        return len(self.wafer_unit)

In [6]:
max_len = max(set([len(wafer[1]) for wafer in wafer_list]))

train_dataset = datasetAE(X_train, 70, max_len, cycle_dict)
valid_dataset = datasetAE(X_valid, 70, max_len, cycle_dict)
test_dataset = datasetAE(X_test, 70, max_len, cycle_dict)

train_loader = DataLoader(train_dataset, batch_size=64, drop_last=True)
valid_loader = DataLoader(valid_dataset, batch_size=64, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=25, drop_last=True)

In [54]:
class I_CNN(torch.nn.Module):
    def __init__(self, sensor_num):
        super(I_CNN, self).__init__()

        layer_list = []
        for num in range(sensor_num):
            extract_layer = torch.nn.Sequential(
                torch.nn.Conv1d(in_channels=1, out_channels=256, kernel_size=9, stride=2),
                torch.nn.ReLU(),
                torch.nn.MaxPool1d(kernel_size=3, stride =3),
                torch.nn.Conv1d(in_channels=256, out_channels=256, kernel_size=5, stride=2),
                torch.nn.ReLU(),
                torch.nn.MaxPool1d(kernel_size=3, stride =3),
            )

            layer_list.append(extract_layer)

        self.extract_layer = nn.ModuleList(layer_list)

        self.diagnosis_layer = torch.nn.Sequential(
            torch.nn.Conv1d(in_channels=sensor_num, out_channels=256, kernel_size=512),
            torch.nn.ReLU(),
        )

        self.detection_layer = torch.nn.Sequential(
            torch.nn.Linear(256, 128),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.5),
            torch.nn.Linear(128, 2),
        )

    def forward(self, x):
        
        b, s, t = x.shape
        x_ = x.transpose(1, 0)

        stack_sensor = []
        for idx, x__ in enumerate(x_):
            x__ = x__.unsqueeze(1)
            feature = self.extract_layer[idx](x__)
            flatten = feature.view(b, -1)
            stack_sensor.append(flatten)

        feature_stack = torch.stack(stack_sensor).transpose(0, 1)
        spartial_out = self.diagnosis_layer(feature_stack)
        out = self.detection_layer(spartial_out.squeeze())

        return out

In [55]:
def sparse_group_regul(model, sensor_num, alpha, mode):
    '''
    model : I-CNN model
    sensor_num : number of sensors
    alpha : weights of l1, 12 loss
    mode : t (training mode) , v (validation mode)
    '''
    
    for name, param in model.named_parameters():
        if name == 'diagnosis_layer.0.weight':
            weight = param

    if mode == 't':
        norm_weights = torch.tensor(0.0, requires_grad=True)
    elif mode == 'v':
        norm_weights = torch.tensor(0.0)

    for sensor in range(0, sensor_num):
        sensor_weight = weight[:, sensor, :]
        para_m = np.sqrt(len(sensor_weight.flatten()))

        # L1 part
        l1_loss = (alpha) * torch.norm(sensor_weight, p=1)

        # L2 part
        l2_loss = (1 - alpha) * (para_m) * torch.norm(sensor_weight, p=2)

        norm_weights = norm_weights + (l1_loss + l2_loss)

    return norm_weights

In [56]:
def loss_icnn(model, x, y, criterion, sensor_num, alpha = 0.5, lambda_ = 0.1, mode = 't'):
    '''
    model : I-CNN model
    x : input data
    y : label
    criterion : loss funcion e.g. nn.CrossEntrophy
    sensor_num : number of sensors
    alpha : weights of l1, 12 loss
    mode : t (training mode) , v (validation mode)
    lambda_ : weight of sparse_group_regul_loss
    '''
    
    output = model(x)
    loss_ = criterion(output, y)
    sparse_group_regul_loss = sparse_group_regul(model = model, sensor_num = sensor_num,
                                                 alpha = alpha, mode = mode)
    loss = loss_ + (lambda_ * sparse_group_regul_loss)
    
    return loss

In [57]:
device = 'cpu'
model = I_CNN(7).to(device)
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)